In [1]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
from scipy.stats import entropy
from sklearn.model_selection import train_test_split
from typing import List, Dict, Tuple, Set
import json
from datetime import datetime
from text_augmentation import DataAugmentor
from annotation_manager import AnnotationManager
from sklearn.metrics import accuracy_score, classification_report
import os

In [3]:
class ConsistencyCalculator:
    """Calculates consistency metrics between original and augmented texts"""
    def __init__(self, base_point: float = 1.3):
        self.base_point = base_point  # Similar to β in CALD
        
    def calculate_confidence_weight(self, probs: np.ndarray) -> float:
        """Calculate confidence weight based on prediction probabilities"""
        return float(np.max(probs))
    
    def calculate_prediction_difference(self, orig_probs: np.ndarray, aug_probs: np.ndarray) -> float:
        """Calculate Jensen-Shannon divergence between predictions"""
        m = 0.5 * (orig_probs + aug_probs)
        js_div = 0.5 * (entropy(orig_probs, m) + entropy(aug_probs, m))
        return float(js_div)
    
    def calculate_consistency_score(self, 
                                  original_probs: np.ndarray, 
                                  augmented_probs_list: List[np.ndarray]) -> float:
        """
        Calculate consistency score for a text and its augmentations
        Returns a score where higher values indicate more informative samples
        """
        consistency_scores = []
        
        for aug_probs in augmented_probs_list:
            # Calculate prediction difference
            pred_diff = self.calculate_prediction_difference(original_probs, aug_probs)
            
            # Calculate confidence weights
            orig_conf = self.calculate_confidence_weight(original_probs)
            aug_conf = self.calculate_confidence_weight(aug_probs)
            conf_weight = 0.5 * (orig_conf + aug_conf)
            
            # Combine into single consistency score
            # Subtract from base_point similar to CALD's approach
            consistency = abs(self.base_point - (pred_diff * conf_weight))
            consistency_scores.append(consistency)
        
        # Return average consistency across all augmentations
        return np.mean(consistency_scores)

In [5]:
class SentimentDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[int], tokenizer, max_length: int = 512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)  # Ensure label is long type
        }

In [7]:
class SentimentDistributionCalculator:
    """Calculates and compares sentiment distributions"""
    def __init__(self):
        self.softmax = torch.nn.Softmax(dim=0)
    
    def get_pool_distribution(self, df: pd.DataFrame, indices: Set[int]) -> np.ndarray:
        """Calculate sentiment distribution of a pool of samples"""
        if not indices:
            return np.ones(2) / 2  # Uniform distribution for empty pool
        
        pool_sentiments = df.loc[list(indices)]['sentiment'].value_counts()
        distribution = np.zeros(2)  # Binary sentiment
        distribution[0] = pool_sentiments.get('negative', 0)
        distribution[1] = pool_sentiments.get('positive', 0)
        return distribution / distribution.sum()
    
    def calculate_js_divergence(self, dist1: np.ndarray, dist2: np.ndarray) -> float:
        """Calculate Jensen-Shannon divergence between two distributions"""
        m = 0.5 * (dist1 + dist2)
        js_div = 0.5 * (entropy(dist1, m) + entropy(dist2, m))
        return float(js_div)

In [9]:
class TwoStageActiveLearning:
    def __init__(self, 
                 model_name: str = 'roberta-base',
                 device: str = 'cuda' if torch.cuda.is_available() else 'cpu',
                 batch_size: int = 128,
                 samples_per_iteration: int = 50,
                 total_iterations: int = 3,
                 expansion_ratio: float = 0.2):
        
        self.device = device
        self.batch_size = batch_size
        self.samples_per_iteration = samples_per_iteration
        self.expanded_pool_size = int(samples_per_iteration * (1 + expansion_ratio))
        self.total_iterations = total_iterations
        
        # Initialize tokenizer separately from model
        self.tokenizer = RobertaTokenizer.from_pretrained(model_name)
        self.model = RobertaForSequenceClassification.from_pretrained(model_name)
        self.model.to(device)
        
        # Initialize other components
        self.augmentor = DataAugmentor(combinations=["SBDR", "SBD", "SDR"])
        self.consistency_calc = ConsistencyCalculator()
        self.distribution_calc = SentimentDistributionCalculator()
        
        # Initialize tracking
        self.current_iteration = 0
        self.labeled_indices: Set[int] = set()
        self.history = []
    
    def stage_one_selection(self, 
                          df: pd.DataFrame, 
                          current_labeled_indices: Set[int]) -> List[int]:
        """Stage 1: Select samples based on individual information (consistency)"""
        unlabeled_df = df[~df.index.isin(current_labeled_indices)].copy()
        all_consistency_scores = []
        
        print("Stage 1: Calculating consistency scores...")
        for idx, row in unlabeled_df.iterrows():
            # Get original text predictions
            original_probs = self.get_predictions([row['review']])[0]
            
            # Get augmented versions and their predictions
            augmented_texts = self.augmentor.augment_text(row['review'])
            augmented_probs = self.get_predictions(augmented_texts)
            
            # Calculate consistency score
            consistency_score = self.consistency_calc.calculate_consistency_score(
                original_probs, augmented_probs)
            
            all_consistency_scores.append((idx, consistency_score))
        
        # Sort by consistency score and select top samples for initial pool
        sorted_scores = sorted(all_consistency_scores, key=lambda x: x[1], reverse=True)
        initial_pool_indices = [idx for idx, _ in sorted_scores[:self.expanded_pool_size]]
        
        return initial_pool_indices
    
    def stage_two_selection(self, 
                          df: pd.DataFrame, 
                          initial_pool_indices: List[int]) -> List[int]:
        """Stage 2: Select final samples based on mutual information"""
        print("Stage 2: Calculating mutual information...")
        
        # Get sentiment distribution of labeled pool
        labeled_distribution = self.distribution_calc.get_pool_distribution(
            df, self.labeled_indices)
        
        # Calculate mutual information scores for initial pool
        mutual_info_scores = []
        
        for idx in initial_pool_indices:
            # Get predictions for the sample
            text = df.loc[idx, 'review']
            probs = self.get_predictions([text])[0]
            
            # Create temporary distribution including this sample
            temp_indices = self.labeled_indices.union({idx})
            temp_distribution = self.distribution_calc.get_pool_distribution(
                df, temp_indices)
            
            # Calculate JS divergence
            js_div = self.distribution_calc.calculate_js_divergence(
                labeled_distribution, temp_distribution)
            
            mutual_info_scores.append((idx, js_div))
        
        # Sort by JS divergence (higher means more different from current distribution)
        sorted_scores = sorted(mutual_info_scores, key=lambda x: x[1], reverse=True)
        final_indices = [idx for idx, _ in sorted_scores[:self.samples_per_iteration]]
        
        return final_indices
    
    def run_iteration(self, df: pd.DataFrame) -> Tuple[Set[int], Dict]:
        """Run one complete iteration of two-stage active learning"""
        print(f"\nStarting iteration {self.current_iteration + 1}/{self.total_iterations}")
        
        # Stage 1: Select initial pool based on consistency
        initial_indices = self.stage_one_selection(df, self.labeled_indices)
        
        # Stage 2: Select final samples based on mutual information
        final_indices = self.stage_two_selection(df, initial_indices)
        
        # Update labeled indices
        self.labeled_indices.update(final_indices)
        
        # Record iteration stats
        stats = {
            'iteration': self.current_iteration + 1,
            'initial_pool_size': len(initial_indices),
            'final_selected': len(final_indices),
            'total_labeled': len(self.labeled_indices),
            'timestamp': datetime.now().isoformat()
        }
        self.history.append(stats)
        
        self.current_iteration += 1
        
        return self.labeled_indices, stats
    
    def get_predictions(self, texts: List[str]) -> np.ndarray:
        """Get model predictions for a list of texts"""
        # Create dummy labels (they're not used for prediction)
        dummy_labels = [0] * len(texts)
        
        dataset = SentimentDataset(
            texts=texts,
            labels=dummy_labels,
            tokenizer=self.tokenizer
        )
        
        dataloader = DataLoader(dataset, batch_size=self.batch_size)
        predictions = []
        
        self.model.eval()
        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                
                outputs = self.model(input_ids, attention_mask=attention_mask)
                probs = torch.softmax(outputs.logits, dim=1)
                predictions.extend(probs.cpu().numpy())
        
        return np.array(predictions)

    def save_state(self, filepath: str):
        """Save active learning state"""
        state = {
            'current_iteration': self.current_iteration,
            'labeled_indices': list(self.labeled_indices),
            'history': self.history
        }
        with open(filepath, 'w') as f:
            json.dump(state, f)
    
    def load_state(self, filepath: str):
        """Load active learning state"""
        with open(filepath, 'r') as f:
            state = json.load(f)
        self.current_iteration = state['current_iteration']
        self.labeled_indices = set(state['labeled_indices'])
        self.history = state['history']

In [11]:
def train_model(model, tokenizer, train_data: pd.DataFrame, device: str) -> None:
    """Train model with fixed data types"""
    model.train()
    
    # Create dataset
    dataset = SentimentDataset(
        texts=train_data['review'].tolist(),
        labels=[1 if s == 'positive' else 0 for s in train_data['sentiment']],
        tokenizer=tokenizer
    )
    
    batch_size = 8
    train_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True
    )
    
    # Calculate class weights with float32
    class_counts = train_data['sentiment'].value_counts()
    weights = torch.tensor([
        len(train_data) / (2 * class_counts['negative']),
        len(train_data) / (2 * class_counts['positive'])
    ], dtype=torch.float32).to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
    criterion = torch.nn.CrossEntropyLoss(weight=weights)
    
    num_epochs = 10
    
    for epoch in range(num_epochs):
        total_loss = 0
        all_predictions = []
        all_labels = []
        
        for batch in train_loader:
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits.float()  # Ensure logits are float
            loss = criterion(logits, labels)
            
            # Get predictions
            probs = torch.softmax(logits, dim=1)
            predictions = torch.argmax(logits, dim=1)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
        # End of epoch
        avg_loss = total_loss / len(train_loader)
        accuracy = accuracy_score(all_labels, all_predictions)
        
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print(f"Average Loss: {avg_loss:.4f}")
        print(f"Accuracy: {accuracy:.4f}")

In [13]:
def prepare_training_data(original_df: pd.DataFrame, annotations: Dict[str, str], augmentor: DataAugmentor) -> pd.DataFrame:
    """Prepare training data with controlled augmentation"""
    training_data = []
    
    # Process each annotated sample
    for idx, sentiment in annotations.items():
        # Get original text
        original_text = original_df.loc[int(idx), 'review']
        
        # Add original sample
        training_data.append({
            'review': original_text,
            'sentiment': sentiment,
            'is_augmented': False,
            'original_idx': idx
        })
        
        # Get augmented versions - using just one combination for controlled augmentation
        augmented_texts = augmentor.augment_text(original_text)[:4]  # Limit to 4 augmentations
        
        # Add augmented samples with same sentiment
        for aug_text in augmented_texts:
            training_data.append({
                'review': aug_text,
                'sentiment': sentiment,
                'is_augmented': True,
                'original_idx': idx
            })
    
    # Convert to DataFrame
    training_df = pd.DataFrame(training_data)
    
    print("\nTraining Data Statistics:")
    print(f"Original samples: {len(annotations)}")
    print(f"Total samples after augmentation: {len(training_df)}")
    print("\nSentiment Distribution:")
    print(training_df['sentiment'].value_counts())
    
    return training_df

In [15]:
class TestDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[int], tokenizer, max_length: int = 512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }


In [17]:
def evaluate_model(model, dataset: TestDataset, device: str):
    """Evaluate model with input monitoring"""
    model.eval()
    dataloader = DataLoader(dataset, batch_size=8)
    
    all_predictions = []
    all_labels = []
    unique_logits = set()  # Track unique logits
    
    print("\nStarting evaluation...")
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            
            # Track unique logits
            batch_logits_tuple = tuple(map(tuple, logits.cpu().numpy()))
            unique_logits.update(batch_logits_tuple)
            
            probs = torch.softmax(logits, dim=1)
            predictions = torch.argmax(logits, dim=1)
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            if batch_idx < 3:  # Print first 3 batches
                print(f"\nBatch {batch_idx + 1}:")
                for i in range(len(predictions)):
                    print(f"Sample {i}:")
                    print(f"  Logits: {logits[i]}")
                    print(f"  Probabilities: {probs[i]}")
                    print(f"  Prediction: {predictions[i]}, True label: {labels[i]}")
    
    print(f"\nNumber of unique logit pairs: {len(unique_logits)}")
    
    accuracy = accuracy_score(all_labels, all_predictions)
    report = classification_report(all_labels, all_predictions, target_names=['Negative', 'Positive'])
    
    return accuracy, report

In [19]:
# Load dataset
df = pd.read_csv('datasets/imdb_dataset_sampled_300.csv')
    
# Initialize active learning
active_learner = TwoStageActiveLearning(
    samples_per_iteration=50,
    total_iterations=3,
    expansion_ratio=0.2
)
    
# Initialize annotation manager
annotator = AnnotationManager(
    dataset_path='datasets/imdb_dataset_sampled_300.csv',
    state_path='active_learning_state.json'
)
    
# Create directory for model checkpoints
os.makedirs('model_checkpoints', exist_ok=True)
    
# Run active learning iterations
for iteration in range(active_learner.total_iterations):
    print(f"\n{'='*80}")
    print(f"Starting Iteration {iteration + 1}/{active_learner.total_iterations}")
    print(f"{'='*80}")
        
    # 1. Run active learning selection
    labeled_indices, stats = active_learner.run_iteration(df)
    active_learner.save_state('active_learning_state.json')
        
    print(f"\nSelection Statistics:")
    print(f"Initial pool size: {stats['initial_pool_size']}")
    print(f"Final selected samples: {stats['final_selected']}")
    print(f"Total samples needing annotation: {stats['total_labeled']}")
        
    # 2. Collect human annotations
    print("\nStarting annotation session...")
        
    while True:
        # Get current annotation status
        current_annotations = annotator.get_all_annotations()
        pending_indices = set(int(idx) for idx in labeled_indices) - set(int(idx) for idx in current_annotations.keys())
            
        if not pending_indices:
            print("All samples have been annotated!")
            break
                
        print(f"\n{len(pending_indices)} samples need annotation.")
        choice = input("Enter 'a' to annotate, 'q' to quit: ").lower()
            
        if choice == 'q':
            print("\nExiting without completing annotations. Please run the program again to continue.")
        elif choice == 'a':
            annotator.run_annotation_session_for_indices(list(pending_indices))
        else:
            print("Invalid choice. Please enter 'a' or 'q'.")
        
    # 3. Get all annotations and prepare training data
    all_annotations = annotator.get_all_annotations()

    # Prepare training data with augmentations
    training_data = prepare_training_data(
        original_df=df,
        annotations=all_annotations,
        augmentor=active_learner.augmentor
    )
        
    # 4. Train model on annotated data
    print("\nRetraining model on annotated data...")
    train_model(
        model=active_learner.model,
        tokenizer=active_learner.tokenizer,
        train_data=training_data,
        device=active_learner.device
    )
        
    # 5. Save model checkpoint
    checkpoint_path = f'model_checkpoints/model_iteration_{iteration + 1}.pt'
    torch.save(active_learner.model.state_dict(), checkpoint_path)
    print(f"\nSaved model checkpoint to {checkpoint_path}")
        
    if iteration < active_learner.total_iterations - 1:
        print("\nPress Enter to continue to next iteration...")
        input()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Starting Iteration 1/3

Starting iteration 1/3
Stage 1: Calculating consistency scores...


Token indices sequence length is longer than the specified maximum sequence length for this model (904 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (907 > 512). Running this sequence through the model will result in indexing errors


Stage 2: Calculating mutual information...

Selection Statistics:
Initial pool size: 60
Final selected samples: 50
Total samples needing annotation: 50

Starting annotation session...

50 samples need annotation.


Enter 'a' to annotate, 'q' to quit:  a


...


Review #119:
--------------------------------------------------------------------------------
Before launching into whether this film is worth your time or not, I should inform you I've never seen another adaptation of Carmen, so if you're looking for a review on how it ranks amongst others, this might not be of much use to you.<br /><br />The only time I've come across Carmen was on the car stereo when driving through Spain on a family holiday when I was a teenager. I didn't pay much attention to it because I didn't like opera at the time and I didn't know any better. The story has been around for 150 years or so. Do I feel I've missed out after seeing this movie? Yes, mainly due to the plot, but also because if all the actresses who played Carmen looked like Paz Vega, I would have all the adaptations happily sitting in my DVD collection.<br /><br />Directed by Vicente Aranda (who also co-rewrote the story with Joaquim Jordà), the story is told through the eyes of the original author

Sentiment (p/n/q):  n



Review #121:
--------------------------------------------------------------------------------
If I guess your "palabra", will you let me go through?- Asks William Geld, a Tim Robbins that keeps on acting like if they told him a fantastic joke and he is attempting not to laugh.<br /><br />He is trying to get to a forbidden area. The woman stopping him continues blabbering: -Your "palabra" is Carrefour.<br /><br />-How did you know?- the lady asks, surprised. He answers, in the name of Wisdom: -I was hearing when you weren't talking. :/<br /><br />Yes, this defines the movie. This precisely. It doesn't matter if Carrefour is "road conjunction" in french, or if the Future is coldly bureaucratic and mixes languages. Or if Samantha Morton has nothing, nothing of Spanish (Maria Gonzalez being her name in the film) with her Irish, Scotish whatever tone.<br /><br />It's boring and dull. If you fall in believing there are multiple symbolisms, you will buy the most bizarre, sickening love relat

Sentiment (p/n/q):  n


Saved 50 annotations to annotations\session_20241205_110349_annotations.json

Progress: 150/50 samples annotated
All samples have been annotated!

Training Data Statistics:
Original samples: 150
Total samples after augmentation: 750

Sentiment Distribution:
sentiment
positive    425
negative    325
Name: count, dtype: int64

Retraining model on annotated data...

Epoch 1/10
Average Loss: 0.6965
Accuracy: 0.5467

Epoch 2/10
Average Loss: 0.7069
Accuracy: 0.4893

Epoch 3/10
Average Loss: 0.6980
Accuracy: 0.5027

Epoch 4/10
Average Loss: 0.6963
Accuracy: 0.5120

Epoch 5/10
Average Loss: 0.6979
Accuracy: 0.4827

Epoch 6/10
Average Loss: 0.6988
Accuracy: 0.5320

Epoch 7/10
Average Loss: 0.7022
Accuracy: 0.4813

Epoch 8/10
Average Loss: 0.7004
Accuracy: 0.4667

Epoch 9/10
Average Loss: 0.7017
Accuracy: 0.5107

Epoch 10/10
Average Loss: 0.6974
Accuracy: 0.5160

Saved model checkpoint to model_checkpoints/model_iteration_3.pt
